# Finding most-widely used, multi-contextual words


To build the third function, "Context Detector", I will extract the word-sense associatoin from Bert model. 
But I certainly do not want to serach for contextual meaning for every possible word that a user can possibly use. Therefore, in this notebook, I will come up with a dictionary of words that Enron employees commonly use and potentially in different contexts. To do so, I will use email topic labels, which were hand-coded by CMU students (available from https://data.world/brianray/enron-email-dataset)


* 1) Load libraries and define functions
* 2) Import data: email data and labeld data 
* 3) Calculate TF-IDF scores for each words in the company-wide email corpus by selecting words that occur frequently, across many people's 
* 4) Using the topic labels data, calculate topic-level term frequency
    * Join the labeled data with TF-IDF results
* 5) Build and store a dictionary of words that widely-used (according to TF-IDF scores) and multi-contextual words (that appear across all categories, quiet frequently).



## 1. Loading libraries and defining functions

In [1]:
import pandas as pd
import numpy as np
from subprocess import check_output
from nltk.tokenize.regexp import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer 
from nltk.util import ngrams 
from nltk.probability import FreqDist
import os, re, nltk, string

def email_clean(text):
    text = re.sub(r'\n--.*?\n', '', text, flags=re.DOTALL)
    text = re.sub(r'enron.com', '', text, flags=re.DOTALL)
    text = re.sub(r'Forwarded by.*?Subject:', '', text, flags=re.DOTALL) 
    text = re.sub(r'Fwd:.*?Subject:', '', text, flags=re.DOTALL) 
    text = re.sub(r'Fw:.*?Subject:', '', text, flags=re.DOTALL)     
    text = re.sub(r'FW:.*?Subject:', '', text, flags=re.DOTALL)         
    text = re.sub(r'Forwarded:.*?Subject:', '', text, flags=re.DOTALL)         
    text = re.sub(r'From:.*?Subject:', '', text, flags=re.DOTALL)
    text = re.sub(r'PM', '', text, flags=re.DOTALL)
    text = re.sub(r'AM', '', text, flags=re.DOTALL)
    
    return text

def clean(text):
    stop = set(stopwords.words('english'))
    stop.update(("to","cc","subject","http","from","sent",
                 "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa", 
                 "enron america corp", "enron", "etc", "na"))
    exclude = set(string.punctuation) 
    lemma = WordNetLemmatizer()
    porter= PorterStemmer()
    
    text=text.rstrip()
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    stop_free = " ".join([i for i in text.lower().split() if((i not in stop) and (not i.isdigit()))])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    #stem = " ".join(porter.stem(token) for token in normalized.split())
    
    return normalized

## 2. Importing data 

In [ ]:
path_to_email_data = 'C:/Users/Margeum/Dropbox/DS projects/05. Email data/emails_in_csv'
os.chdir(path_to_email_data)
emails_df = pd.read_csv('emails_parsed.csv')
labeled_emails_df = pd.read_csv("enron_05_17_2015_with_labels_v2.csv") # Labeled data 
address_user_df = pd.read_csv('./address_user_df.csv')

## 3. Calculating TF-IDF scores 

Calculate word frequencies for the email corpus 

In [ ]:
tf_list = []

for index, row in address_user_df.iterrows():
    
    text_cleaned_i = []

    user_i = row['user']
    print user_i
    
    lastname = row['address'].split('@')[0].split('.')[-1]
    firstname = str(row['address'].split('.')[0])

    text_to_clean_df_i = emails_df[emails_df["user"] == user_i][["content", "user"]].reset_index()
        
    for text in text_to_clean_df_i['content']:
        text_cleaned_i.append(clean(email_clean(text)).split())

    unlisted_text_cleaned_i = [item for sublist in text_cleaned_i for item in sublist]
    freqdist_user_i = nltk.FreqDist(ngrams(unlisted_text_cleaned_i, 1))

    tf_list.append(freqdist_user_i)    


Calculating word frequencies at the user-account level (Documet-level frequencies where all emails in a user's account are treated as a document) 

In [ ]:
idf_text_cleaned_i = []

for text in emails_df["content"]:
    idf_text_cleaned_i.append(clean(email_clean(text)).split())

idf_unlisted_text_cleaned_i = [item for sublist in idf_text_cleaned_i for item in sublist]
idf_list = nltk.FreqDist(ngrams(idf_unlisted_text_cleaned_i, 1))

df_idf = pd.DataFrame.from_dict(idf_list, orient='index')
df_idf.columns = ['Frequency']
df_idf.index.name = 'Term'
df_idf.sort_values(by = "Frequency", ascending = False)
#df_idf.head()

Now merge the corpus-level frequencies and account-level frequencies by using word as key. Then, calculate the TF-IDF scores.

In [ ]:
 tf_idf_stacked = df_idf

for i in range(len(address_user_df)):
    my_td = pd.DataFrame.from_dict(tf_list[i], orient='index')
    my_td.columns = ['Frequency']
    my_td.index.name = 'Term'

    tf_idf_i = my_td.reset_index().set_index('Term').join(df_idf.reset_index().set_index('Term'), on= 'Term', how='left',
                                              lsuffix='_left', rsuffix='_right')

    tf_idf_i["idf"] =  np.log(tf_idf_i['Frequency_right'])/(len(df_idf)+1)
    tf_idf_i["tf_idf"] = tf_idf_i['Frequency_left']*tf_idf_i['idf']
    tf_idf_i.sort_values(by = "tf_idf", ascending = False)

    tf_idf_i = pd.DataFrame(tf_idf_i["tf_idf"])
    tf_idf_i.columns = tf_idf_i.columns + "_" + str(i)

    tf_idf_stacked = tf_idf_stacked.join(tf_idf_i, on = 'Term', how ='left', lsuffix='_left', rsuffix  ='right')


col_list= list(tf_idf_stacked)
col_list.remove('Frequency')


tf_idf_stacked['mean'] = tf_idf_stacked[col_list].mean(axis=1)
tf_idf_stacked['variance'] = tf_idf_stacked[col_list].var(axis=1)
tf_idf_stacked['count_notnull'] = tf_idf_stacked[col_list].count(axis=1)
tf_idf_stacked.sort_values(by = "mean", ascending = False).head(10)


In [20]:
print (str(len(tf_idf_stacked["mean"])) + ' unique words are in the email corpus')

522762 unique words are in the email corpus


We need to cut the words from this complete list. First, we will keep only top 1 percent words in terms of mean TF-IDF score. In other words, we will keep words with high importance in terms of their frequency across different accounts.

In [24]:
mean_val_cut_99 = tf_idf_stacked["mean"].quantile(.99)
df_mean_val_cut_99 = tf_idf_stacked[tf_idf_stacked['mean'] > mean_val_cut_99]

print (str(len(df_mean_val_cut_99)) + ' unique words are in this list of top 1 percent words)')
print ('Top 1 percent words account for ' + 
       str(float(df_mean_val_cut_99["Frequency"].sum())/float(tf_idf_stacked["Frequency"].sum())) +
       ' unique words are in this list of top 1 percent words)' + '% in terms of frequency')

5156 unique words are in this list of top 1 percent words)
Top 1 percent words account for 0.798830791283 unique words are in this list of top 1 percent words)% in terms of frequency


In [29]:
df_mean_val_cut_99.sort_values(by = "mean", ascending = False).head(10)

,Frequency,tf_idf_0,tf_idf_1,tf_idf_2,tf_idf_3,tf_idf_4,tf_idf_5,tf_idf_6,tf_idf_7,tf_idf_8,...,tf_idf_130,tf_idf_131,tf_idf_132,tf_idf_133,tf_idf_134,tf_idf_135,tf_idf_136,mean,variance,count_notnull
Term,,,,,,,,,,,,,,,,,,,,,
"(com,)",664513,0.064602,0.136463,0.035238,0.040803,0.002539,0.536490,0.230609,0.045009,0.041880,...,0.064295,0.027775,0.041008,0.075271,0.031134,0.030083,0.006822,0.121391,0.054427,137
"(ect,)",595113,0.051099,0.114941,0.002162,0.004349,0.000076,0.283041,0.022942,0.842460,0.000153,...,0.013989,0.049141,0.000127,0.011700,0.007758,0.003001,NaN,0.109993,0.067947,134
"(please,)",379986,0.042960,0.069430,0.014132,0.014648,0.009904,0.093490,0.057190,0.243459,0.049375,...,0.021529,0.045836,0.014574,0.029836,0.028927,0.022144,0.009462,0.065944,0.010998,137
"(e,)",378364,0.030563,0.067662,0.011867,0.027738,0.002580,0.136676,0.055550,0.097857,0.021375,...,0.029532,0.018377,0.008624,0.027591,0.021989,0.012677,0.006756,0.065836,0.022772,137
"(would,)",325382,0.035473,0.071772,0.008231,0.011096,0.003156,0.080464,0.023600,0.194751,0.042077,...,0.030666,0.018186,0.010902,0.008887,0.012868,0.019618,0.005609,0.056304,0.018686,137
"(power,)",309093,0.019926,0.049984,0.005755,0.024375,0.002442,0.010519,0.041472,0.068072,0.011486,...,0.025996,0.043382,0.009769,0.006045,0.009262,0.015549,0.005634,0.053380,0.047698,137
"(hou,)",273226,0.024497,0.050358,0.001461,0.002490,NaN,0.157157,0.010776,0.302318,0.000623,...,0.007567,0.020162,NaN,0.004837,0.003712,0.000479,NaN,0.052184,0.012847,122
"(energy,)",298781,0.010515,0.073026,0.002870,0.021464,0.006077,0.010877,0.022019,0.054794,0.020162,...,0.031497,0.007524,0.005740,0.016279,0.011624,0.009888,0.003087,0.051578,0.030421,137
"(company,)",288719,0.010054,0.122422,0.005604,0.010342,0.005147,0.015994,0.022055,0.072010,0.030377,...,0.084517,0.005772,0.005243,0.041657,0.007793,0.007312,0.001756,0.049604,0.025251,137


The last column in the df_mean_val_cut_99 dataframe, It shows that some of these top-1-percent words appear only in some of the accounts. These words might be important for those user who use them, but less likely to be a multi-context words. Therefore, I will remove them from the list. 

In [35]:
count_null_cut = 0.5
print (str(count_null_cut*100) + '% of words have more than ' + str(df_mean_val_cut_99["count_notnull"].quantile(count_null_cut)) + ' accounts')

50.0% of words have more than 105.0 accounts


Although 50% is an arbitrary cut, we will use that as our cut point for now. 

In [39]:
keywords_dictionary = df_mean_val_cut_99[df_mean_val_cut_99["count_notnull"]> df_mean_val_cut_99["count_notnull"].quantile(count_null_cut)]
print ('We now have ' + str(len(keywords_dictionary)) + ' words in the list')

We now have 2571 words in the list


In [ ]:
#len(keywords_dictionary)
#keywords_dictionary.to_csv('keywords_dictionary.csv', index=True)

## 4. Count topic-level word frequencies 

Now, I want to figure out whether the key words in our dictionary appears across differen topic areas. The labeled_emails_df has email topic labels, but only for a subset of emails. Using message IDs, I will merge the labeles to our keywords dictionary so that we can further trim down the dictionary.  

In [ ]:
labeled_emails_df.head()
#labeled_emails_df["labeled"].describe()
labeled_messages = labeled_emails_df[labeled_emails_df["labeled"]==True]
print ('The dataset provides topic labels for ' + str(len(labeled_messages)) + ' emails')
#labeled_messages.head()

From the label information, we will use "primary topics", which corresponds to level_1 == 3 and level_2 ranges from 1 to 13. (Full description of the topic labels are available here: https://data.world/brianray/enron-email-dataset). I focus on primary topics because they are relevant to the company's business and strategies (as opposed to personal emails or administrative/editing/etc.).

In [53]:
# Currently, Cat_1 implies that the first category that a human coder identified. Our category of interest, "3" can appear any of the 12 columns: Cat_1_level1, Cat_2_level1, ... Cat_12_level1. 
# So we will extract "3"s from the 12 columns. If 3 exists in any of the level 1 columns, we also want to extract the sub-category(i.e., level 2 category) from from the associated level_2 column.

labeled_messages_red = labeled_messages[(labeled_messages["Cat_1_level_1"].fillna(0.0).astype(int) == 3) | 
                        (labeled_messages["Cat_2_level_1"].fillna(0.0).astype(int) == 3) |
                        (labeled_messages["Cat_3_level_1"].fillna(0.0).astype(int) == 3) |
                        (labeled_messages["Cat_4_level_1"].fillna(0.0).astype(int) == 3) |
                        (labeled_messages["Cat_5_level_1"].fillna(0.0).astype(int) == 3) |
                        (labeled_messages["Cat_6_level_1"].fillna(0.0).astype(int) == 3) |
                        (labeled_messages["Cat_7_level_1"].fillna(0.0).astype(int) == 3) |
                        (labeled_messages["Cat_8_level_1"].fillna(0.0).astype(int) == 3) |
                        (labeled_messages["Cat_9_level_1"].fillna(0.0).astype(int) == 3) |
                        (labeled_messages["Cat_10_level_1"].fillna(0.0).astype(int) == 3) |
                        (labeled_messages["Cat_11_level_1"].fillna(0.0).astype(int) == 3) |
                        (labeled_messages["Cat_12_level_1"].fillna(0.0).astype(int) == 3) 
                       ]
print (str(len(labeled_messages_red)) + ' messages are labeled as primary topics')

879 messages are labeled as primary topics


In [82]:
cat_columns = ["Cat_1_level_1", "Cat_1_level_2", 
              "Cat_2_level_1", "Cat_2_level_2",
              "Cat_3_level_1", "Cat_3_level_2", 
              "Cat_4_level_1", "Cat_4_level_2", 
              "Cat_5_level_1", "Cat_5_level_2", 
              "Cat_6_level_1", "Cat_6_level_2", 
              "Cat_7_level_1", "Cat_7_level_2", 
              "Cat_8_level_1", "Cat_8_level_2", 
              "Cat_9_level_1", "Cat_9_level_2", 
              "Cat_10_level_1", "Cat_10_level_2", 
              "Cat_11_level_1", "Cat_11_level_2", 
              "Cat_12_level_1", "Cat_12_level_2", 
              ]
#cat_columns
labeled_messages_red[cat_columns] = labeled_messages_red[cat_columns].fillna(value=0).astype(int)


In [88]:
label_dict_dt = pd.DataFrame.from_dict(label_dict, orient = 'index', columns = ["topic", "content"])

In [107]:
topic_3_1_list = []
topic_3_2_list = []
topic_3_3_list = []
topic_3_4_list = []
topic_3_5_list = []
topic_3_6_list = []
topic_3_7_list = []
topic_3_8_list = []
topic_3_9_list = []
topic_3_10_list = []
topic_3_11_list = []
topic_3_12_list = []

for i in range(len(label_dict_dt)): 
    if '3.1' in label_dict_dt.iloc[i]['topic']:
        topic_3_1_list.append(label_dict_dt.iloc[i]['content'])
    if '3.2' in label_dict_dt.iloc[i]['topic']:
        topic_3_2_list.append(label_dict_dt.iloc[i]['content'])
    if '3.3' in label_dict_dt.iloc[i]['topic']:
        topic_3_3_list.append(label_dict_dt.iloc[i]['content'])
    if '3.4' in label_dict_dt.iloc[i]['topic']:
        topic_3_4_list.append(label_dict_dt.iloc[i]['content'])
    if '3.5' in label_dict_dt.iloc[i]['topic']:
        topic_3_5_list.append(label_dict_dt.iloc[i]['content'])
    if '3.6' in label_dict_dt.iloc[i]['topic']:
        topic_3_6_list.append(label_dict_dt.iloc[i]['content'])
    if '3.7' in label_dict_dt.iloc[i]['topic']:
        topic_3_7_list.append(label_dict_dt.iloc[i]['content'])
    if '3.8' in label_dict_dt.iloc[i]['topic']:
        topic_3_8_list.append(label_dict_dt.iloc[i]['content'])
    if '3.9' in label_dict_dt.iloc[i]['topic']:
        topic_3_9_list.append(label_dict_dt.iloc[i]['content'])
    if '3.10' in label_dict_dt.iloc[i]['topic']:
        topic_3_10_list.append(label_dict_dt.iloc[i]['content'])
    if '3.11' in label_dict_dt.iloc[i]['topic']:
        topic_3_11_list.append(label_dict_dt.iloc[i]['content'])
    if '3.12' in label_dict_dt.iloc[i]['topic']:
        topic_3_12_list.append(label_dict_dt.iloc[i]['content'])

For words in our key word dictionary, let's count the occurance in eachof the topic document. 

In [181]:
topic_3_1_str = ' '.join(topic_3_1_list).lower()
topic_3_2_str = ' '.join(topic_3_2_list).lower()
topic_3_3_str = ' '.join(topic_3_3_list).lower()
topic_3_4_str = ' '.join(topic_3_4_list).lower()
topic_3_5_str = ' '.join(topic_3_5_list).lower()
topic_3_6_str = ' '.join(topic_3_6_list).lower()
topic_3_7_str = ' '.join(topic_3_7_list).lower()
topic_3_8_str = ' '.join(topic_3_8_list).lower()
topic_3_9_str = ' '.join(topic_3_9_list).lower()
topic_3_10_str = ' '.join(topic_3_10_list).lower()
topic_3_11_str = ' '.join(topic_3_11_list).lower()
topic_3_12_str = ' '.join(topic_3_12_list).lower()

In [182]:
topic_bin_3_1 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_1_str: 
        topic_bin_3_1.append(1)
    else: 
        topic_bin_3_1.append(0)

        
topic_bin_3_2 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_2_str: 
        topic_bin_3_2.append(1)
    else: 
        topic_bin_3_2.append(0)


topic_bin_3_3 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_3_str: 
        topic_bin_3_3.append(1)
    else: 
        topic_bin_3_3.append(0)
        

topic_bin_3_4 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_4_str: 
        topic_bin_3_4.append(1)
    else: 
        topic_bin_3_4.append(0)
        

topic_bin_3_5 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_5_str: 
        topic_bin_3_5.append(1)
    else: 
        topic_bin_3_5.append(0)
        

topic_bin_3_6 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_6_str: 
        topic_bin_3_6.append(1)
    else: 
        topic_bin_3_6.append(0)
        

topic_bin_3_7 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_7_str: 
        topic_bin_3_7.append(1)
    else: 
        topic_bin_3_7.append(0)
        

topic_bin_3_8 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_8_str: 
        topic_bin_3_8.append(1)
    else: 
        topic_bin_3_8.append(0)
        

topic_bin_3_9 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_9_str: 
        topic_bin_3_9.append(1)
    else: 
        topic_bin_3_9.append(0)
        

topic_bin_3_10 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_10_str: 
        topic_bin_3_10.append(1)
    else: 
        topic_bin_3_10.append(0)
        

topic_bin_3_11 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_11_str: 
        topic_bin_3_11.append(1)
    else: 
        topic_bin_3_11.append(0)
        

topic_bin_3_12 = []
for i in range(len(keywords_dictionary)):
    word = list((keywords_dictionary.index[i]))[0]
    if word in topic_3_12_str: 
        topic_bin_3_12.append(1)
    else: 
        topic_bin_3_12.append(0)
          

In [ ]:
keywords_dictionary['topic_bin_3_1']= topic_bin_3_1
keywords_dictionary['topic_bin_3_2']= topic_bin_3_2
keywords_dictionary['topic_bin_3_3']= topic_bin_3_3
keywords_dictionary['topic_bin_3_4']= topic_bin_3_4
keywords_dictionary['topic_bin_3_5']= topic_bin_3_5
keywords_dictionary['topic_bin_3_6']= topic_bin_3_6
keywords_dictionary['topic_bin_3_7']= topic_bin_3_7
keywords_dictionary['topic_bin_3_8']= topic_bin_3_8
keywords_dictionary['topic_bin_3_9']= topic_bin_3_9
keywords_dictionary['topic_bin_3_10']= topic_bin_3_10
keywords_dictionary['topic_bin_3_11']= topic_bin_3_11
keywords_dictionary['topic_bin_3_12']= topic_bin_3_12

topic_cat_list = ['topic_bin_3_1', 'topic_bin_3_2', 'topic_bin_3_3', 
                 'topic_bin_3_4', 'topic_bin_3_5', 'topic_bin_3_6',
                 'topic_bin_3_7', 'topic_bin_3_8', 'topic_bin_3_9', 
                 'topic_bin_3_10', 'topic_bin_3_11', 'topic_bin_3_12']

keywords_dictionary['count_topics'] = keywords_dictionary[topic_cat_list].sum(axis=1)
#keywords_dictionary['count_topics'].describe()

In [200]:
print ('Among ' + str(len(keywords_dictionary)) +
       ' words in the keyword dictionary, ' + 
       str(len(keywords_dictionary[keywords_dictionary['count_topics'] == 12])) +
       ' appear in all of the primary category')

Among 2571 words in the keyword dictionary, 1168 appear in all of the primary category


Let's keep those 1168 words as our keywords of interest. 

In [ ]:
keywords_dictionary = keywords_dictionary[keywords_dictionary['count_topics'] == 12][['Frequency', 'mean']]

In [275]:
keywords_dictionary.sort_values(by='Frequency', ascending=False).head(20)

,Frequency,mean
Term,,
"(com,)",664513,0.121391
"(ect,)",595113,0.109993
"(please,)",379986,0.065944
"(e,)",378364,0.065836
"(would,)",325382,0.056304
"(power,)",309093,0.053380
"(energy,)",298781,0.051578
"(company,)",288719,0.049604
"(new,)",275770,0.047004


Let's further trim down the list by eliminating common English stopwords

In [219]:
my_stopwords = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", 
                "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually",
                "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", 
                "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", 
                "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", 
                "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart",
                "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't",
                "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", 
                "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", 
                "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", 
                "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", 
                "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu",
                "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause",
                "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", 
                "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", 
                "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", 
                "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", 
                "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", 
                "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", 
                "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", 
                "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", 
                "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", 
                "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", 
                "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", 
                "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", 
                "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", 
                "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", 
                "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", 
                "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", 
                "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", 
                "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes",
                "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", 
                "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", 
                "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", 
                "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", 
                "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", 
                "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", 
                "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km",
                "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb",
                "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", 
                "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", 
                "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", 
                "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", 
                "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", 
                "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", 
                "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", 
                "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", 
                "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", 
                "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", 
                "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", 
                "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", 
                "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", 
                "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", 
                "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", 
                "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", 
                "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", 
                "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", 
                "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", 
                "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", 
                "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", 
                "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", 
                "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", 
                "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", 
                "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", 
                "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", 
                "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", 
                "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", 
                "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", 
                "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", 
                "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
                "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", 
                "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", 
                "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", 
                "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", 
                "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", 
                "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", 
                "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", 
                "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", 
                "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", 
                "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", 
                "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", 
                "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", 
                "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", 
                "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", 
                "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", 
                "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", 
                "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", 
                "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]


In [310]:
my_vocab = []

for item in list(keywords_dictionary.index):
    my_vocab.append(str(item).split('\'')[1])

new_dict = keywords_dictionary[~keywords_dictionary['my_vocab'].isin(my_stopwords)]

Further trim down the list by excluding names, Enron-specific abbreviations, and days/months/etc.

In [330]:
custom_stopwords = ['john', 'jeff', 'vince', 'david', 'steve', 'smith', 'chris', 
'michael', 'richard', 'bush', 'paul', 'jim', 'susan', 'joe', 
'tom', 'duke', 'rick', 'eric', 'dan', 'steven', 'ben', 'lisa', 
'frank', 'carol', 'karen', 'lee',  'gary', 'tim', 'martin', 
'rob', 'sue', 'allen', 'stephen', 'linda', 'hey', 'matt', 
'sarah', 'ted', 'harry', 'barbara', 'stanley', 'anderson', 'pat', 
'jane', 'louis', 'phil', 'jean', 'stan', 
'day', 'year', 'morning', 'today', 'yesterday', 'tomorrow', 
'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 
'januray', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december',
'jan', 'dec', 'nov', 'fri', 'oct', 'sept', 'mar', 'apr', 'aug', 'jun', 'sep', 'li', 
'ect', 'hou', 'fax', 'ena', 'asp', 'san', 'iso', 'pg', 'eb', 'dow', 'mp', 'mw', 'gov',  'ceo', 'fyi', 
'mm', 'cal', 'usa', 'calif', 'pre', 'tel', 'ene', 'ew', 'var', 'pa', 
'ext', 'ets', 'rep', 'ha', 'fed', 'sea', 'mi', 'min', 
'sw', 'ing', 'isc', 'edi', 'ferc','doc','corp', 'north', 'america']

my_custom_vocab = []

for item in list(new_dict.index):
    my_custom_vocab.append(str(item).split('\'')[1])

#new_dict['my_custom_vocab'] = my_custom_vocab
reduced_dict = new_dict[~new_dict['my_custom_vocab'].isin(custom_stopwords)]
reduced_dict = reduced_dict.sort_values(by='mean')

In [331]:
reduced_dict[50:100]
len(reduced_dict)

711

## 5. Store the key word dictionary 

In [332]:
#final_keywords_dictionary.to_csv('final_keywords_dictionary.csv', index=True)
reduced_dict.to_csv('keywords_reduced.csv', index=True) 